# Dataset Analysis 5: Repeated Holdout

In [1]:
import pandas as pd
import numpy as np
from datasets import *
from training import *

from sklearn import svm
from sklearn.preprocessing import StandardScaler

Normal generated NA, max censoring

In [2]:
dogs = load_skl_dogs_2016(NApolicy='normal', censoringPolicy='max', scaler=StandardScaler())
dogs.data.shape

/home/elvis/Dropbox/Università/tesi/svm-thesis/experiments/datasets/dogs_2006_2016.py:99: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  data.set_value(i, "Survival time", survmax)
/home/elvis/Dropbox/Università/tesi/svm-thesis/experiments/datasets/dogs_2006_2016.py:118: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  data.set_value(i, nacol, np.random.normal(loc=params[nacol][0], scale=params[nacol][1]))


(161, 16)

In [4]:
X, y = dogs.data, dogs.target
param_grid = [
  {'C': [0.5, 1, 2, 4, 8, 16], 'epsilon':[0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10], 'gamma': ['auto'], 'degree': [0], 'coef0':[0], 'kernel': ['linear']},
  {'C': [0.5, 1, 2, 4, 8, 16], 'epsilon':[0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'degree': [0], 'coef0':[0], 'kernel': ['rbf']},
  {'C': [0.5, 1, 2, 4, 8, 16], 'epsilon':[0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'degree': [1,2,3], 'coef0': [-100, -10, -1, 0, 1, 10, 100], 'kernel': ['poly']}
 ]

best_result = (0,-np.inf)

for i in range(10):
    curr_result = SVR_gridsearch_holdout(X, y, svm.SVR, param_grid, 20, 25)
    if curr_result[1] > best_result[1]:
        best_result = curr_result

print(best_result)

((1, -1, 2, 10, 0.1, 'poly'), -0.019972097986667103)


In [5]:
mean_score = 0
for i in range(10):
    X_Train, X_Test, y_Train, y_Test = train_test_split(X, y, test_size=25)
    best_params = best_result[0]
    best_svr = svm.SVR(C=best_params[0], coef0=best_params[1], degree=best_params[2], epsilon=best_params[3], gamma=best_params[4], kernel=best_params[5])
    best_svr.fit(X_Train, y_Train)
    mean_score += best_svr.score(X_Test, y_Test)/10
print(mean_score)

-0.32082424485918504


In [19]:
dogs = load_skl_dogs_2016(NApolicy='normal', censoringPolicy='drop', scaler=StandardScaler())
dogs.data.shape

(118, 16)

In [5]:
X, y = dogs.data, dogs.target
param_grid = [
  {'C': [0.5, 1, 2, 4, 8, 16], 'epsilon':[0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10], 'gamma': ['auto'], 'degree': [0], 'coef0':[0], 'kernel': ['linear']},
  {'C': [0.5, 1, 2, 4, 8, 16], 'epsilon':[0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'degree': [0], 'coef0':[0], 'kernel': ['rbf']},
  {'C': [0.5, 1, 2, 4, 8, 16], 'epsilon':[0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'degree': [1,2,3], 'coef0': [-100, -10, -1, 0, 1, 10, 100], 'kernel': ['poly']}
 ]

best_result = (0,-np.inf)

for i in range(10):
    curr_result = SVR_gridsearch_holdout(X, y, svm.SVR, param_grid, 10, 15)
    if curr_result[1] > best_result[1]:
        best_result = curr_result

print(best_result)

((4, -10, 3, 10, 0.01, 'poly'), 0.10298839102452439)


In [23]:
mean_score = 0
for i in range(10):
    X_Train, X_Test, y_Train, y_Test = train_test_split(X, y, test_size=25)
    best_params = best_result[0]
    best_svr = svm.SVR(C=best_params[0], coef0=best_params[1], degree=best_params[2], epsilon=best_params[3], gamma=best_params[4], kernel=best_params[5])
    best_svr.fit(X_Train, y_Train)
    mean_score += best_svr.score(X_Test, y_Test)/10
print(mean_score)

-0.0826718279464
